# Data

> Datasets and collators for Extreme Classification

In [ ]:
#| default_exp data

In [ ]:
#| hide
%load_ext autoreload 
%autoreload 2

In [ ]:
#| export
import torch, inspect, numpy as np, pandas as pd, torch.nn.functional as F, random

from scipy import sparse
from itertools import chain
from tqdm.auto import tqdm

from IPython.display import display
from typing import Dict, Optional, Callable
from torch.utils.data import Dataset,DataLoader
from xclib.data import data_utils as du
from xclib.utils.sparse import retain_topk
from transformers import PreTrainedTokenizerBase, AutoTokenizer

from fastcore.utils import *
from fastcore.meta import *
from fastcore.dispatch import *

from xcai.core import *

In [ ]:
#| hide
from xcai.transform import *

In [ ]:
#| hide
from nbdev.showdoc import *
import nbdev; nbdev.nbdev_export()

## Data

In [ ]:
dset_dir = f'/home/scai/phd/aiz218323/Projects/XC/data/(mapped)LF-WikiSeeAlsoTitles-320K'
data_cfg = {
    'info_column_names': ['identifier', 'input_text'],
    'use_tokenizer': True,
    'tokenizer': 'bert-base-uncased',
    'tokenization_column': 'input_text',
    'main_max_data_sequence_length': 32,
    'main_max_lbl_sequence_length': 32,
    'meta_max_sequence_length': 64,
    'padding': True,
    'return_tensors': 'pt',
}

In [ ]:
#| export
def _read_sparse_file(fname:str):
    if fname.endswith('.txt'): return du.read_sparse_file(fname)
    elif fname.endswith('.npz'): return sparse.load_npz(fname)
    else: raise ValueError(f'Invalid file extension : {fname}')
    

In [ ]:
#| export
class MainXCData:
    
    @classmethod
    @delegates(Info.from_txt)
    def from_file(cls, data_lbl:str, data_info:str, lbl_info:str, data_lbl_filterer:Optional[str]=None, 
                  main_max_data_sequence_length:Optional[int]=None, main_max_lbl_sequence_length:Optional[int]=None, **kwargs):
        return {
            'data_lbl': _read_sparse_file(data_lbl),
            'data_info': Info.from_txt(data_info, max_sequence_length=main_max_data_sequence_length, **kwargs),
            'lbl_info': Info.from_txt(lbl_info, max_sequence_length=main_max_lbl_sequence_length, **kwargs),
            'data_lbl_filterer': Filterer.load_filter(data_lbl_filterer),
        }
    

In [ ]:
train_cfg = {
    'data_lbl': f'{dset_dir}/trn_X_Y.txt',
    'data_info': f'{dset_dir}/raw_data/train.raw.txt',
    'lbl_info': f'{dset_dir}/raw_data/label.raw.txt',
    'data_lbl_filterer': f'{dset_dir}/filter_labels_train.txt',
}
train_data = MainXCData.from_file(**train_cfg, **data_cfg)

In [ ]:
#| export
class MetaXCData:
    
    @classmethod
    @delegates(Info.from_txt)
    def from_file(cls, data_meta:str, lbl_meta:str, meta_info:str, prefix:str, meta_max_sequence_length:Optional[int]=None, **kwargs):
        return {
            'prefix': prefix,
            'data_meta': _read_sparse_file(data_meta),
            'lbl_meta': _read_sparse_file(lbl_meta),
            'meta_info': Info.from_txt(meta_info, max_sequence_length=meta_max_sequence_length, **kwargs),
        }
    

In [ ]:
meta_cfg = {
    'prefix': 'hlk',
    'data_meta': f'{dset_dir}/hyper_link_trn_X_Y.txt',
    'lbl_meta': f'{dset_dir}/hyper_link_lbl_X_Y.txt',
    'meta_info': f'{dset_dir}/raw_data/hyper_link.raw.txt',
}
meta_data = MetaXCData.from_file(**meta_cfg, **data_cfg)

/home/scai/phd/aiz218323/.local/lib/python3.9/site-packages/xclib-0.97-py3.9-linux-x86_64.egg/xclib/data/data_utils.py:263: UserWarning: Header mis-match from inferred shape!
  warnings.warn("Header mis-match from inferred shape!")


## Dataset

### Helper functions

In [ ]:
def permute_elements(data):
    
  if isinstance(data, list):
    data_copy = data.copy()
    random.shuffle(data_copy)
    return data_copy

  elif isinstance(data, np.ndarray):
    indices = np.random.permutation(len(data))
    return data[indices]
      
  elif isinstance(data, torch.Tensor):
    indices = torch.randperm(len(data))
    return data[indices]
  else:
    raise TypeError("Unsupported data type. Expected list, NumPy array, or PyTorch tensor.")


### `BaseXCDataset`

In [ ]:
#| export
class BaseXCDataset(Dataset):
    def __init__(self):
        self.n_data, self.n_lbl, self.n_meta, self.n_samples = None, None, None, None

    def __len__(self):
        return self.n_data if self.n_data is not None else 0

    def splitter(self, valid_pct:Optional[float]=0.2, seed=None):
        if seed is not None: torch.manual_seed(seed)
        rnd_idx = list(torch.randperm(self.n_data).numpy())
        cut = int(valid_pct * self.n_data)
        train, valid = self._getitems(rnd_idx[cut:]), self._getitems(rnd_idx[:cut])
        return train, valid

    def sample(self, pct:Optional[float]=0.2, n:Optional[int]=None, seed=None):
        if seed is not None: torch.manual_seed(seed)
        rnd_idx = list(torch.randperm(self.n_data).numpy())
        cut = int(pct * self.n_data) if n is None else max(1, n)
        return self._getitems(rnd_idx[:cut])
        
    def _verify_info(self, info:Dict):
        if info is None: raise ValueError('`info` cannot be empty.')
        n_info = [len(v) for k,v in info.items()]
        if len(n_info) == 0 or n_info[0] == 0: raise ValueError('`info` cannot be empty.')
        if np.all([n_info[0] == o for o in n_info]) == False: 
            raise ValueError('All `data_info` fields should have equal number of elements.')
        return n_info[0]
        
    def show_data(self, n:Optional[int]=10, seed:Optional[int]=None):
        if n < 1: return
        if seed: np.random.seed(seed)
        idx = np.random.permutation(self.n_data)[:n]
        d = [self[i] for i in idx]
        df = pd.DataFrame({k:[o[k] for o in d] for k in d[0]})
        with pd.option_context('display.max_colwidth', None, 'display.max_columns', None):
            display(df)

    def prune_data_lbl(self, data_lbl:sparse.csr_matrix, data_repr:torch.Tensor, lbl_repr:torch.Tensor, batch_size:Optional[int]=64, 
                       thresh:Optional[float]=0.1, topk:Optional[int]=None):
        data_repr,lbl_repr = F.normalize(data_repr, dim=1), F.normalize(lbl_repr, dim=1)
        curr_data_lbl = data_lbl.copy()
        rows, cols = data_lbl.nonzero()
        dl = DataLoader(list(zip(rows, cols)), batch_size=batch_size, shuffle=False)
        score = None
        for b in tqdm(dl, total=len(dl)): 
            sc = data_repr[b[0]].unsqueeze(1)@lbl_repr[b[1]].unsqueeze(2)
            sc = sc.squeeze()
            sc = torch.where(sc < thresh, 0, sc)
            score = sc if score is None else torch.hstack([score, sc])
        curr_data_lbl.data[:] = score
        curr_data_lbl.eliminate_zeros()
        if topk is not None: 
            curr_data_lbl = retain_topk(curr_data_lbl, k=topk)
        return curr_data_lbl

    def get_info(self, prefix, idxs, info, info_keys):
        x = dict()
        for k,v in info.items():
            if k in info_keys:
                if isinstance(v, np.ndarray) or isinstance(v, torch.Tensor):
                    o = v[idxs]
                    if isinstance(o, np.ndarray): o = torch.from_numpy(o)
                    x[f'{prefix}_{k}'] = o
                else:
                    x[f'{prefix}_{k}'] = [v[idx] for idx in idxs]
        return x

    def extract_items(self, prefix:str, data_lbl:List, idxs:List, n_samples:int, n_s_samples:int, oversample:bool, 
                      info:Dict, info_keys:List):
        x, entity = dict(), prefix.split('2')[-1]
        
        x[f'p{prefix}_idx'] = [data_lbl[idx] for idx in idxs]
        if n_samples: 
            x[f'p{prefix}_idx'] = [[o[i] for i in np.random.permutation(len(o))[:n_samples]] for o in x[f'p{prefix}_idx']]
        x[f'p{prefix}_{entity}2ptr'] = torch.tensor([len(o) for o in x[f'p{prefix}_idx']], dtype=torch.int64)

        if oversample:
            x[f'{prefix}_idx'] = [np.random.choice(o, size=n_s_samples) if len(o) else [] for o in x[f'p{prefix}_idx']]
        else:
            x[f'{prefix}_idx'] = [[o[i] for i in np.random.permutation(len(o))[:n_s_samples]] for o in x[f'p{prefix}_idx']]
        x[f'{prefix}_{entity}2ptr'] = torch.tensor([len(o) for o in x[f'{prefix}_idx']], dtype=torch.int64)
        
        x[f'{prefix}_idx'] = torch.tensor(list(chain(*x[f'{prefix}_idx'])), dtype=torch.int64)
        x[f'p{prefix}_idx'] = torch.tensor(list(chain(*x[f'p{prefix}_idx'])), dtype=torch.int64)
        
        if info is not None:
            x.update(self.get_info(prefix, x[f'{prefix}_idx'], info, info_keys))
            
        return x
            

### `MainXCDataset`

In [ ]:
#| export
class MainXCDataset(BaseXCDataset):
    def __init__(self,
                 data_info:Dict,
                 data_lbl:Optional[sparse.csr_matrix]=None,
                 lbl_info:Optional[Dict]=None,
                 data_lbl_filterer:Optional[Union[sparse.csr_matrix,np.array]]=None,
                 n_lbl_samples:Optional[int]=None,
                 data_info_keys:Optional[List]=None,
                 lbl_info_keys:Optional[List]=None,
                 **kwargs):
        super().__init__()
        store_attr('data_info,data_lbl,lbl_info,data_lbl_filterer,n_lbl_samples,data_info_keys,lbl_info_keys')
        self.curr_data_lbl = None
        
        self._verify_inputs()
        self._store_indices()
        
    @classmethod
    @delegates(MainXCData.from_file)
    def from_file(cls, n_lbl_samples:Optional[int]=None, data_info_keys:Optional[List]=None, lbl_info_keys:Optional[List]=None, **kwargs):
        return cls(**MainXCData.from_file(**kwargs), n_lbl_samples=n_lbl_samples, 
                   data_info_keys=data_info_keys, lbl_info_keys=lbl_info_keys)

    def _store_indices(cls):
        if cls.data_lbl is not None: cls.curr_data_lbl = [o.indices.tolist() for o in cls.data_lbl]

    def _verify_inputs(cls):
        cls.n_data = cls._verify_info(cls.data_info)
        if cls.data_info_keys is None: cls.data_info_keys = list(cls.data_info.keys())
        if cls.data_lbl is not None:
            if cls.n_data != cls.data_lbl.shape[0]:
                raise ValueError(f'`data_info`({cls.n_data}) should have same number of datapoints as `data_lbl`({cls.data_lbl.shape[0]})')
            cls.n_lbl = cls.data_lbl.shape[1]
            if cls.lbl_info is not None:
                n_lbl = cls._verify_info(cls.lbl_info)
                if n_lbl != cls.data_lbl.shape[1]:
                    raise ValueError(f'`lbl_info`({n_lbl}) should have same number of labels as `data_lbl`({cls.data_lbl.shape[1]})')
                if cls.lbl_info_keys is None: cls.lbl_info_keys = list(cls.lbl_info.keys())
        

In [ ]:
#| export
@patch
def __getitem__(cls:MainXCDataset, idx:int):
    x = {f'data_{k}': v[idx] for k,v in cls.data_info.items() if k in cls.data_info_keys}
    x['data_idx'] = idx
    if cls.n_lbl is not None:
        prefix = 'lbl2data'
        x[f'{prefix}_idx'] = cls.curr_data_lbl[idx]
        if cls.n_lbl_samples: x[f'{prefix}_idx'] = [x[f'{prefix}_idx'][i] for i in np.random.permutation(len(x[f'{prefix}_idx']))[:cls.n_lbl_samples]]
        if cls.lbl_info is not None:
            x.update({f'{prefix}_{k}':[v[i] for i in x[f'{prefix}_idx']] for k,v in cls.lbl_info.items() if k in cls.lbl_info_keys})
    return x
    

In [ ]:
@patch
def __getitems__(cls:MainXCDataset, idxs:List):
    x = {f'data_{k}':[v[idx] for idx in idxs] for k,v in cls.data_info.items() if k in cls.data_info_keys}
    x['data_idx'] = idxs
    if cls.n_lbl is not None:
        prefix = 'lbl2data'
        x[f'{prefix}_idx'] = [cls.curr_data_lbl[idx] for idx in idxs]
        if cls.n_lbl_samples: x[f'{prefix}_idx'] = [[o[i] for i in np.random.permutation(len(o))[:cls.n_lbl_samples]] for o in x[f'{prefix}_idx']]
        x[f'{prefix}_data2ptr'] = [len(o) for o in x[f'{prefix}_idx']]
        x[f'{prefix}_idx'] = list(chain(*x[f'{prefix}_idx']))
        if cls.lbl_info is not None:
            x.update({f'{prefix}_{k}':[v[i] for i in x[f'{prefix}_idx']] for k,v in cls.lbl_info.items() if k in cls.lbl_info_keys})
    return x
        

In [ ]:
#| export
@patch
def _getitems(cls:MainXCDataset, idxs:List):
    return MainXCDataset(
        {k:[v[idx] for idx in idxs] for k,v in cls.data_info.items()}, 
        data_lbl=cls.data_lbl[idxs] if cls.data_lbl is not None else None, 
        lbl_info=cls.lbl_info, 
        data_lbl_filterer=Filterer.sample(cls.data_lbl_filterer, sz=cls.data_lbl.shape, idx=idxs) if cls.data_lbl_filterer is not None else None,
        n_lbl_samples=cls.n_lbl_samples,
        data_info_keys=cls.data_info_keys,
        lbl_info_keys=cls.lbl_info_keys,
    )

#### Example

In [ ]:
train_main = MainXCDataset(**train_data, n_lbl_samples=2, data_info_keys=[], lbl_info_keys=[])

In [ ]:
train_main.show_data(n=3)

,data_idx,lbl2data_idx
0,611718,"[277518, 208935]"
1,591793,[120489]
2,515058,[111990]


In [ ]:
split1, split2 = train_main.splitter()
split1.n_data, split2.n_data, split1.n_data+split2.n_data, train_main.n_data

(554466, 138616, 693082, 693082)

In [ ]:
sample = train_main.sample(n=30)
sample.n_data, sample.data_lbl, sample.data_lbl_filterer

(30,
 <30x312330 sparse matrix of type '<class 'numpy.float32'>'
 	with 63 stored elements in Compressed Sparse Row format>,
 array([[     1,   1730],
        [     2,  31395],
        [     3,  84824],
        [    14, 270724],
        [    20, 287893]], dtype=int32))

In [ ]:
train_main = MainXCDataset.from_file(**train_cfg, **data_cfg)

In [ ]:
train_main.show_data(n=3, seed=10)

,data_identifier,data_input_text,data_input_ids,data_token_type_ids,data_attention_mask,lbl2data_idx,lbl2data_identifier,lbl2data_input_text,lbl2data_input_ids,lbl2data_token_type_ids,lbl2data_attention_mask
0,Certificate_of_Formula_Compliance,Certificate of Formula Compliance,"[101, 8196, 1997, 5675, 12646, 102]","[0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1]",[244514],[Certificate_of_Origin],[Certificate of Origin],"[[101, 8196, 1997, 4761, 102]]","[[0, 0, 0, 0, 0]]","[[1, 1, 1, 1, 1]]"
1,2017_Sunwolves_season,2017 Sunwolves season,"[101, 2418, 3103, 12155, 20899, 2161, 102]","[0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1]","[163257, 308041]","[Sunwolves, 2017_Super_Rugby_season]","[Sunwolves, 2017 Super Rugby season]","[[101, 3103, 12155, 20899, 102], [101, 2418, 3565, 4043, 2161, 102]]","[[0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]]","[[1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]"
2,Hadza_people,Hadza people,"[101, 2018, 4143, 2111, 102]","[0, 0, 0, 0, 0]","[1, 1, 1, 1, 1]","[8411, 164636, 230647]","[Twa_peoples, Aka_people, Bushmen]","[Twa peoples, Aka people, Bushmen]","[[101, 1056, 4213, 7243, 102], [101, 9875, 2111, 102], [101, 5747, 3549, 102]]","[[0, 0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]]","[[1, 1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1]]"


### `MetaXCDataset`

In [ ]:
#| export
class MetaXCDataset(BaseXCDataset):

    def __init__(self,
                 prefix:str,
                 data_meta:sparse.csr_matrix, 
                 lbl_meta:sparse.csr_matrix, 
                 meta_info:Optional[Dict]=None, 
                 n_data_meta_samples:Optional[int]=None,
                 n_lbl_meta_samples:Optional[int]=None,
                 meta_info_keys:Optional[List]=None,
                 **kwargs):
        store_attr('prefix,data_meta,lbl_meta,meta_info,n_data_meta_samples,n_lbl_meta_samples,meta_info_keys')
        self.curr_data_meta,self.curr_lbl_meta = None,None
        self._verify_inputs()
        self._store_indices()

    def prune_data_meta(self, data_repr:torch.Tensor, meta_repr:torch.Tensor, batch_size:Optional[int]=64, thresh:Optional[float]=0.0, 
                        topk:Optional[int]=None):
        data_meta = self.prune_data_lbl(self.data_meta, data_repr, meta_repr, batch_size, thresh, topk)
        self.curr_data_meta = [o.indices.tolist() for o in data_meta]

    def prune_lbl_meta(self, lbl_repr:torch.Tensor, meta_repr:torch.Tensor, batch_size:Optional[int]=64, thresh:Optional[float]=0.0, 
                       topk:Optional[int]=None):
        lbl_meta = self.prune_data_lbl(self.lbl_meta, lbl_repr, meta_repr, batch_size, thresh, topk)
        self.curr_lbl_meta = [o.indices.tolist() for o in lbl_meta]

    def _store_indices(self):
        if self.data_meta is not None: self.curr_data_meta = [o.indices.tolist() for o in self.data_meta]
        if self.lbl_meta is not None: self.curr_lbl_meta = [o.indices.tolist() for o in self.lbl_meta]

    def update_meta_matrix(self, data_meta:sparse.csr_matrix, lbl_meta:sparse.csr_matrix):
        self.data_meta, self.lbl_meta = data_meta, lbl_meta
        self._store_indices()
        
    def _getitems(self, idxs:List):
        return MetaXCDataset(self.prefix, self.data_meta[idxs], self.lbl_meta, self.meta_info, 
                             self.n_data_meta_samples, self.n_lbl_meta_samples, self.meta_info_keys)

    def _sample_meta_items(self, idxs:List):
        assert max(idxs) < self.n_meta, f"indices should be less than {self.n_meta}"
        meta_info = {k: [v[i] for i in idxs] for k,v in self.meta_info.items()}
        return MetaXCDataset(self.prefix, self.data_meta[:, idxs], self.lbl_meta[:, idxs], meta_info, 
                             self.n_data_meta_samples, self.n_lbl_meta_samples, self.meta_info_keys)
        
    @classmethod
    @delegates(MetaXCData.from_file)
    def from_file(cls, n_data_meta_samples:Optional[int]=None, n_lbl_meta_samples:Optional[int]=None, meta_info_keys:Optional[List]=None, **kwargs):
        return cls(**MetaXCData.from_file(**kwargs), n_data_meta_samples=n_data_meta_samples, 
                   n_lbl_meta_samples=n_lbl_meta_samples, meta_info_keys=meta_info_keys)

    @typedispatch
    def get_lbl_meta(self, idx:int):
        prefix = f'{self.prefix}2lbl2data'
        x = {f'{prefix}_idx': self.curr_lbl_meta[idx]}
        if self.n_lbl_meta_samples: x[f'{prefix}_idx'] = [x[f'{prefix}_idx'][i] for i in np.random.permutation(len(x[f'{prefix}_idx']))[:self.n_lbl_meta_samples]]
        if self.meta_info is not None:
            x.update({f'{prefix}_{k}':[v[i] for i in x[f'{prefix}_idx']] for k,v in self.meta_info.items() if k in self.meta_info_keys})
        return x
    
    @typedispatch
    def get_lbl_meta(self, idxs:List):
        prefix = f'{self.prefix}2lbl2data'
        x = {f'{prefix}_idx': [self.curr_lbl_meta[idx] for idx in idxs]}
        if self.n_lbl_meta_samples: x[f'{prefix}_idx'] = [[o[i] for i in np.random.permutation(len(o))[:self.n_lbl_meta_samples]] for o in x[f'{prefix}_idx']]
        if self.meta_info is not None:
            x.update({f'{prefix}_{k}':[[v[i] for i in o] for o in x[f'{prefix}_idx']] for k,v in self.meta_info.items() if k in self.meta_info_keys})
        return x
        
    def get_data_meta(self, idx:int):
        prefix = f'{self.prefix}2data'
        x = {f'{prefix}_idx': self.curr_data_meta[idx]}
        if self.n_data_meta_samples: x[f'{prefix}_idx'] = [x[f'{prefix}_idx'][i] for i in np.random.permutation(len(x[f'{prefix}_idx']))[:self.n_data_meta_samples]]
        if self.meta_info is not None:
            x.update({f'{prefix}_{k}':[v[i] for i in x[f'{prefix}_idx']] for k,v in self.meta_info.items() if k in self.meta_info_keys})
        return x

    def shape(self):
        return (self.n_data, self.n_lbl, self.n_meta)
        
    def show_data(self, is_lbl:Optional[bool]=False, n:Optional[int]=10, seed:Optional[int]=None):
        if n < 1: return
        if seed: np.random.seed(seed)
        idx = np.random.permutation(self.n_lbl if is_lbl else self.n_data)[:n]
        d = [self.get_lbl_meta(int(i)) for i in idx] if is_lbl else [self.get_data_meta(i) for i in idx]
        df = pd.DataFrame({k:[o[k] for o in d] for k in d[0]})
        with pd.option_context('display.max_colwidth', None):
            display(df)
    

If metadata is available for the main task, then there will be `data_meta` and `lbl_meta`.

In [ ]:
#| export
@patch
def _verify_inputs(cls:MetaXCDataset):
    cls.n_data,cls.n_meta = cls.data_meta.shape[0],cls.data_meta.shape[1]
    
    if cls.lbl_meta is not None:
        cls.n_lbl = cls.lbl_meta.shape[0]
        if cls.lbl_meta.shape[1] != cls.n_meta:
            raise ValueError(f'`lbl_meta`({cls.lbl_meta.shape[1]}) should have same number of columns as `data_meta`({cls.n_meta}).')

    if cls.meta_info is not None:
        n_meta = cls._verify_info(cls.meta_info)
        if n_meta != cls.n_meta:
            raise ValueError(f'`meta_info`({n_meta}) should have same number of entries as number of columns of `data_meta`({cls.n_meta})')
        if cls.meta_info_keys is None: cls.meta_info_keys = list(cls.meta_info.keys())
            

#### Example

In [ ]:
train_meta = MetaXCDataset(**meta_data, n_data_meta_samples=2, n_lbl_meta_samples=2, meta_info_keys=[])

In [ ]:
train_meta.show_data(n=3)

,hlk2data_idx
0,"[368197, 174586]"
1,"[703287, 866329]"
2,"[2932, 191361]"


__sample metadata using indices__

In [ ]:
idxs = np.random.permutation(train_meta.n_meta)[:50]
meta_dset = train_meta._sample_meta_items(idxs)

In [ ]:
meta_dset.data_meta

<693082x50 sparse matrix of type '<class 'numpy.float32'>'
	with 118 stored elements in Compressed Sparse Row format>

In [ ]:
split1, split2 = train_meta.splitter()
split1.shape(), split2.shape()

((554466, 312330, 2458399), (138616, 312330, 2458399))

In [ ]:
sample = train_meta.sample(n=40)
sample.n_data, sample.shape()

(40, (40, 312330, 2458399))

In [ ]:
train_meta.show_data(n=3, is_lbl=True)

,hlk2lbl2data_idx
0,[241864]
1,"[248077, 693197]"
2,"[506790, 57704]"


In [ ]:
train_meta.shape()

(693082, 312330, 2458399)

In [ ]:
train_meta = MetaXCDataset.from_file(**meta_cfg, **data_cfg)

/home/scai/phd/aiz218323/.local/lib/python3.9/site-packages/xclib-0.97-py3.9-linux-x86_64.egg/xclib/data/data_utils.py:263: UserWarning: Header mis-match from inferred shape!
  warnings.warn("Header mis-match from inferred shape!")


In [ ]:
train_meta.show_data(n=3)

,hlk2data_idx,hlk2data_identifier,hlk2data_input_text,hlk2data_input_ids,hlk2data_token_type_ids,hlk2data_attention_mask
0,"[38, 122, 254, 281, 292, 697, 698, 701, 767, 2504, 2754, 2764, 2808, 3223, 3600, 4083, 4322, 4502, 4503, 5203, 19555, 26950, 35767, 45880, 57613, 189987, 247697, 260730, 262600, 280189, 297163, 297313, 361107, 416836, 471498, 619765, 666342, 719698, 818704, 1677742, 2037687, 2414194, 2414195, 2414196]","[Philippines, The_New_York_Times, United_States, France, Forbes, Netherlands, Colombia, Australia, Canada, South_Korea, Japan, Taiwan, New_York_Post, Thailand, Vice_president, Singapore, Food_and_Drug_Administration, Russell_2000_Index, Public_company, Los_Angeles_Times, Internal_Revenue_Service, Certified_Public_Accountant, Time_(magazine), Deseret_News, The_Salt_Lake_Tribune, Barry_Minkow, Chief_financial_officer, UniversitÃ©_de_MontrÃ©al, Therapeutic_Goods_Administration, Chief_operating_officer, U.S._Securities_and_Exchange_Commission, National_Business_Review, Naval_Postgraduate_School, San_Diego_Reader, Commerce_Commission, NSF_International, Public_relations_officer, United_States_Pharmacopeia, USANA_Amphitheatre, Herb_Greenberg, MonaVie, Fraud_Discovery_Institute, Denis_Waitley, Myron_W._Wentz]","[Philippines, The New York Times, United States, France, Forbes, Netherlands, Colombia, Australia, Canada, South Korea, Japan, Taiwan, New York Post, Thailand, Vice president, Singapore, Food and Drug Administration, Russell 2000 Index, Public company, Los Angeles Times, Internal Revenue Service, Certified Public Accountant, Time (magazine), Deseret News, The Salt Lake Tribune, Barry Minkow, Chief financial officer, UniversitÃ© de MontrÃ©al, Therapeutic Goods Administration, Chief operating officer, U.S. Securities and Exchange Commission, National Business Review, Naval Postgraduate School, San Diego Reader, Commerce Commission, NSF International, Public relations officer, United States Pharmacopeia, USANA Amphitheatre, Herb Greenberg, MonaVie, Fraud Discovery Institute, Denis Waitley, Myron W. Wentz]","[[101, 5137, 102], [101, 1996, 2047, 2259, 2335, 102], [101, 2142, 2163, 102], [101, 2605, 102], [101, 10822, 102], [101, 4549, 102], [101, 7379, 102], [101, 2660, 102], [101, 2710, 102], [101, 2148, 4420, 102], [101, 2900, 102], [101, 6629, 102], [101, 2047, 2259, 2695, 102], [101, 6504, 102], [101, 3580, 2343, 102], [101, 5264, 102], [101, 2833, 1998, 4319, 3447, 102], [101, 5735, 2456, 5950, 102], [101, 2270, 2194, 102], [101, 3050, 3349, 2335, 102], [101, 4722, 6599, 2326, 102], [101, 7378, 2270, 17907, 102], [101, 2051, 1006, 2932, 1007, 102], [101, 4078, 7869, 2102, 2739, 102], [101, 1996, 5474, 2697, 10969, 102], [101, 6287, 8117, 24144, 102], [101, 2708, 3361, 2961, 102], [101, 4895, 16402, 28032, 2050, 29652, 2139, 18318, 2527, 29652, 2389, 102], [101, 17261, 5350, 3447, 102], [101, 2708, 4082, 2961, 102], [101, 1057, 1012, 1055, 1012, 12012, 1998, 3863, 3222, 102], [101, 2120, 2449, 3319, 102], [101, 3987, 15438, 2082, 102], [101, 2624, 5277, 8068, 102], [101, 6236, 3222, 102], [101, 24978, 2546, 2248, 102], [101, 2270, 4262, 2961, 102], [101, 2142, 2163, 6887, 27292, 22684, 5051, 2401, 102], [101, 3915, 2532, 23713, 16584, 20192, 7913, 102], [101, 12810, 24190, 102], [101, 13813, 13469, 102], [101, 9861, 5456, 2820, 102], [101, 11064, 3524, 3051, 102], [101, 2026, 4948, 1059, 1012, 2253, 2480, 102]]","[[0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0], [0, 0, 0, 0], [0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0

### `XCDataset`

In [ ]:
#| export
class MetaXCDatasets(dict):

    def __init__(self, meta:Dict):
        super().__init__(meta)
        for o in meta: setattr(self, o, meta[o])

    def __setitem__(self, key, value):
        super().__setitem__(key, value)
        setattr(self, key, value)

    def __delitem__(self, key):
        super().__delitem__(key)
        delattr(self, key)
        

In [ ]:
#| export
class XCDataset(BaseXCDataset):

    def __init__(self, data:MainXCDataset, **kwargs):
        super().__init__()
        self.data, self.meta = data, MetaXCDatasets({k:kwargs[k] for k in self.get_meta_args(**kwargs) if isinstance(kwargs[k], MetaXCDataset)})
        self._verify_inputs()

    @staticmethod
    def get_meta_args(**kwargs):
        return [k for k in kwargs if re.match(r'.*_meta$', k)]
        
    def _getitems(self, idxs:List):
        return XCDataset(self.data._getitems(idxs), **{k:meta._getitems(idxs) for k,meta in self.meta.items()})
        
    @classmethod
    @delegates(MainXCDataset.from_file)
    def from_file(cls, **kwargs):
        data = MainXCDataset.from_file(**kwargs)
        meta_kwargs = {o:kwargs.pop(o) for o in cls.get_meta_args(**kwargs)}
        meta = {k:MetaXCDataset.from_file(**v, **kwargs) for k,v in meta_kwargs.items()}
        return cls(data, **meta)

    def _verify_inputs(self):
        self.n_data, self.n_lbl = self.data.n_data, self.data.n_lbl
        if len(self.meta):
            self.n_meta = len(self.meta)
            for meta in self.meta.values():
                if meta.n_data != self.n_data: 
                    raise ValueError(f'`meta`({meta.n_data}) and `data`({self.n_data}) should have the same number of datapoints.')
                if self.n_lbl is not None and meta.n_lbl != self.n_lbl: 
                    raise ValueError(f'`meta`({meta.n_lbl}) and `data`({self.n_lbl}) should have the same number of labels.')


    def __getitem__(self, idx:int):
        x = self.data[idx]
        if self.n_meta:
            for m in self.meta.values():
                x.update(m.get_data_meta(idx))
                if self.n_lbl: x.update(m.get_lbl_meta(x['lbl2data_idx']))
        return x

    @property
    def lbl_info(self): return self.data.lbl_info

    @property
    def lbl_dset(self): return MainXCDataset(self.data.lbl_info)

    @property
    def data_info(self): return self.data.data_info

    @property
    def data_dset(self): return MainXCDataset(self.data.data_info) 

    def one_batch(self, bsz:Optional[int]=10, seed:Optional[int]=None):
        if seed is not None: torch.manual_seed(seed)
        idxs = list(torch.randperm(len(self)).numpy())[:bsz]
        return [self[idx] for idx in idxs]

    def _retain_randk(self, matrix:sparse.csr_matrix, topk:Optional[int]=3):
        data, indices, indptr = [], [], np.zeros_like(matrix.indptr)
        for i,row in tqdm(enumerate(matrix), total=matrix.shape[0]):
            if row.nnz > 0:
                idx = np.random.randint(row.nnz, size=topk)
                ind, d = row.indices[idx], row.data[idx]    
            else:
                ind, d = np.arange(topk), np.zeros(topk)
                
            indptr[i+1] = indptr[i] + topk
            indices.append(ind); data.append(d)
        data = np.hstack(data)
        indices = np.hstack(indices)
        o = sparse.csr_matrix((data, indices, indptr), shape=matrix.shape, dtype=matrix.dtype)
        o.sort_indices()
        return o

    def _remove_data(self, meta_1:sparse.csr_matrix, meta_2:sparse.csr_matrix, pct:Optional[float]=0.3):
        n_data = min(len(meta_1.data),len(meta_2.data))
        n = int(n_data * pct)
        idx = np.random.permutation(n_data)
        idx_1,idx_2 = idx[:n], idx[n:]
        meta_1.data[idx_1] = 0; meta_1.eliminate_zeros()
        meta_2.data[idx_2] = 0; meta_2.eliminate_zeros()

    def _mix_meta_matrix(self, meta_1:sparse.csr_matrix, meta_2:sparse.csr_matrix, pct:Optional[float]=0.3, k:Optional[int]=3):
        meta_1 = self._retain_randk(meta_1, topk=k)
        meta_2 = self._retain_randk(meta_2, topk=k)
        self._remove_data(meta_1, meta_2, pct)
        return meta_1 + meta_2

    def mix_meta_dataset(self, meta_1:str, meta_2:str, pct:Optional[float]=0.3, k:Optional[int]=3):
        if pct < 1:
            meta_info = self.meta[f'{meta_1}_meta'].meta_info
            data_meta = self._mix_meta_matrix(self.meta[f'{meta_1}_meta'].data_meta, self.meta[f'{meta_2}_meta'].data_meta, pct=pct, k=k)
            lbl_meta = self._mix_meta_matrix(self.meta[f'{meta_1}_meta'].lbl_meta, self.meta[f'{meta_2}_meta'].lbl_meta, pct=pct, k=k)
            self.meta['hyb_meta'] = MetaXCDataset('hyb', data_meta, lbl_meta, meta_info, 
                                                  n_data_meta_samples=self.meta[f'{meta_2}_meta'].n_data_meta_samples,
                                                  n_lbl_meta_samples=self.meta[f'{meta_2}_meta'].n_lbl_meta_samples, 
                                                  meta_info_keys=self.meta[f'{meta_2}_meta'].meta_info_keys)
        else:
            self.meta['hyb_meta'] = MetaXCDataset('hyb', self.meta[f'{meta_2}_meta'].data_meta, 
                                                  self.meta[f'{meta_2}_meta'].lbl_meta, 
                                                  self.meta[f'{meta_2}_meta'].meta_info, 
                                                  n_data_meta_samples=self.meta[f'{meta_2}_meta'].n_data_meta_samples,
                                                  n_lbl_meta_samples=self.meta[f'{meta_2}_meta'].n_lbl_meta_samples, 
                                                  meta_info_keys=self.meta[f'{meta_2}_meta'].meta_info_keys)
       

#### Example

In [ ]:
train_dset = XCDataset(train_main, hlk_meta=train_meta)

In [ ]:
train_dset

<__main__.XCDataset>

In [ ]:
b = train_dset.one_batch(10)

In [ ]:
b

[{'data_identifier': 'Forty-first_Amendment_of_the_Constitution_of_India',
  'data_input_text': 'Forty-first Amendment of the Constitution of India',
  'data_input_ids': [101,
   5659,
   1011,
   2034,
   7450,
   1997,
   1996,
   4552,
   1997,
   2634,
   102],
  'data_token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  'data_attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  'lbl2data_idx': [130717],
  'lbl2data_identifier': ['List_of_amendments_of_the_Constitution_of_India'],
  'lbl2data_input_text': ['List of amendments of the Constitution of India'],
  'lbl2data_input_ids': [[101,
    2862,
    1997,
    16051,
    1997,
    1996,
    4552,
    1997,
    2634,
    102]],
  'lbl2data_token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
  'lbl2data_attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
  'hlk2data_idx': [17688, 1484],
  'hlk2data_identifier': ['Lok_Sabha', 'India'],
  'hlk2data_input_text': ['Lok Sabha', 'India'],
  'hlk2data_input_ids': [[101, 13660, 11200, 102],

In [ ]:
train_dset.show_data(n=2)

,data_identifier,data_input_text,data_input_ids,data_token_type_ids,data_attention_mask,lbl2data_idx,lbl2data_identifier,lbl2data_input_text,lbl2data_input_ids,lbl2data_token_type_ids,lbl2data_attention_mask,hlk2data_idx,hlk2data_identifier,hlk2data_input_text,hlk2data_input_ids,hlk2data_token_type_ids,hlk2data_attention_mask,hlk2lbl2data_idx,hlk2lbl2data_identifier,hlk2lbl2data_input_text,hlk2lbl2data_input_ids,hlk2lbl2data_token_type_ids,hlk2lbl2data_attention_mask
0,Roman_Catholic_Diocese_of_ZamoÅÄ-LubaczÃ³w,Roman Catholic Diocese of ZamoÅÄ-LubaczÃ³w,"[101, 3142, 3234, 5801, 1997, 23564, 5302, 11057, 1011, 11320, 3676, 27966, 2050, 18107, 2860, 102]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[123397, 108938]","[Roman_Catholicism_in_Poland, List_of_Catholic_dioceses_in_Poland]","[Roman Catholicism in Poland, List of Catholic dioceses in Poland]","[[101, 3142, 16138, 1999, 3735, 102], [101, 2862, 1997, 3234, 26586, 1999, 3735, 102]]","[[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0]]","[[1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1]]","[316178, 2329402]","[Podkarpackie_Voivodeship, RadruÅ¼]","[Podkarpackie Voivodeship, RadruÅ¼]","[[101, 17491, 6673, 23947, 2666, 7225, 102], [101, 10958, 13626, 6692, 29664, 102]]","[[0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]]","[[1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]","[[], [484309, 988342]]","[[], [Catholic_Church_in_Poland, Roman_Catholic_Diocese_of_ElblÄ g]]","[[], [Catholic Church in Poland, Roman Catholic Diocese of ElblÄ g]]","[[], [[101, 3234, 2277, 1999, 3735, 102], [101, 3142, 3234, 5801, 1997, 3449, 28522, 2290, 102]]]","[[], [[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0]]]","[[], [[1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1]]]"
1,Tan_Chay_Wa's_tombstone_trial,Tan Chay Wa's tombstone trial,"[101, 9092, 15775, 2100, 11333, 1005, 1055, 26671, 3979, 102]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",[158812],[Chia_Thye_Poh],[Chia Thye Poh],"[[101, 9610, 2050, 15177, 2063, 13433, 2232, 102]]","[[0, 0, 0, 0, 0, 0, 0, 0]]","[[1, 1, 1, 1, 1, 1, 1, 1]]","[273985, 544714]","[Choa_Chu_Kang, Choa_Chu_Kang_Cemetery]","[Choa Chu Kang, Choa Chu Kang Cemetery]","[[101, 16480, 2050, 14684, 16073, 102], [101, 16480, 2050, 14684, 16073, 4528, 102]]","[[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0]]","[[1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1]]","[[31390, 16287]]","[[National_University_of_Singapore, Sentosa]]","[[National University of Singapore, Sentosa]]","[[[101, 2120, 2118, 1997, 5264, 102], [101, 2741, 8820, 102]]]","[[[0, 0, 0, 0, 0, 0], [0, 0, 0, 0]]]","[[[1, 1, 1, 1, 1, 1], [1, 1, 1, 1]]]"


In [ ]:
split1, split2 = train_dset.splitter()
split1.data.n_data, {k:m.shape() for k,m in split1.meta.items()}, split2.data.n_data, {k:m.shape() for k,m in split2.meta.items()}

(554466,
 {'hlk_meta': (554466, 312330, 2458399)},
 138616,
 {'hlk_meta': (138616, 312330, 2458399)})

In [ ]:
sample = train_dset.sample(n=30)
sample.data.n_data, {k:m.shape() for k,m in sample.meta.items()}

(30, {'hlk_meta': (30, 312330, 2458399)})

In [ ]:
train_dset = XCDataset.from_file(**cfg['path']['train'], **cfg['parameters'])

/home/scai/phd/aiz218323/.local/lib/python3.9/site-packages/xclib-0.97-py3.9-linux-x86_64.egg/xclib/data/data_utils.py:263: UserWarning: Header mis-match from inferred shape!
  warnings.warn("Header mis-match from inferred shape!")


In [ ]:
train_dset.show_data(n=2)

,data_identifier,data_input_text,data_input_ids,data_token_type_ids,data_attention_mask,lbl2data_idx,lbl2data_identifier,lbl2data_input_text,lbl2data_input_ids,lbl2data_token_type_ids,lbl2data_attention_mask,hlk2data_idx,hlk2data_identifier,hlk2data_input_text,hlk2data_input_ids,hlk2data_token_type_ids,hlk2data_attention_mask,hlk2lbl2data_idx,hlk2lbl2data_identifier,hlk2lbl2data_input_text,hlk2lbl2data_input_ids,hlk2lbl2data_token_type_ids,hlk2lbl2data_attention_mask
0,Microsail,Microsail,"[101, 12702, 15816, 2140, 102]","[0, 0, 0, 0, 0]","[1, 1, 1, 1, 1]",[194848],[List_of_sailing_boat_types],[List of sailing boat types],"[[101, 2862, 1997, 8354, 4049, 4127, 102]]","[[0, 0, 0, 0, 0, 0, 0]]","[[1, 1, 1, 1, 1, 1, 1]]","[254, 281, 527389, 527392, 883760]","[United_States, France, Gary_Mull, Fractional_rig, Jeanneau]","[United States, France, Gary Mull, Fractional rig, Jeanneau]","[[101, 2142, 2163, 102], [101, 2605, 102], [101, 5639, 14163, 3363, 102], [101, 12884, 2389, 19838, 102], [101, 14537, 4887, 102]]","[[0, 0, 0, 0], [0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0]]","[[1, 1, 1, 1], [1, 1, 1], [1, 1, 1, 1, 1], [1, 1, 1, 1, 1], [1, 1, 1, 1]]","[[43854, 261695, 297082]]","[[National_Collegiate_Athletic_Association, Sacred_Heart_University, Sacred_Heart_Pioneers]]","[[National Collegiate Athletic Association, Sacred Heart University, Sacred Heart Pioneers]]","[[[101, 2120, 9234, 5188, 2523, 102], [101, 6730, 2540, 2118, 102], [101, 6730, 2540, 13200, 102]]]","[[[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]]]","[[[1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1], [1, 1, 1, 1, 1]]]"
1,Captain_Benjamin_Williams_House,Captain Benjamin Williams House,"[101, 2952, 6425, 3766, 2160, 102]","[0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1]",[235682],"[National_Register_of_Historic_Places_listings_in_Middletown,_Connecticut]","[National Register of Historic Places listings in Middletown, Connecticut]","[[101, 2120, 4236, 1997, 3181, 3182, 26213, 1999, 28747, 1010, 6117, 102]]","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]","[7140, 38915]","[National_Register_of_Historic_Places, Connecticut_River]","[National Register of Historic Places, Connecticut River]","[[101, 2120, 4236, 1997, 3181, 3182, 102], [101, 6117, 2314, 102]]","[[0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0]]","[[1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1]]","[[8398, 19705, 26775, 44140, 55989, 64716, 75195, 81592, 129263, 195143, 267626, 281086, 281149, 281203, 281221, 281352, 304511, 316421, 578952, 723622, 742621, 744544, 799315, 844632, 1019297, 1357876, 1357877, 1357878, 1357879, 1357880]]","[[2004_Summer_Olympics, Baseball_at_the_Summer_Olympics, Australians, Run_batted_in, Second_baseman, Maccabi_World_Union, Jewish_Virtual_Library, New_South_Wales_Patriots, Southeastern_Louisiana_University, Sri_Lanka_national_cricket_team, Baseball_at_the_2004_Summer_Olympics, World_Baseball_Classic, Rodney_van_Buizen, Brett_Roneberg, Glenn_Williams, Brendan_Kingman, 2007_Baseball_World_Cup, Greg_Jelks, 2005_Baseball_World_Cup, Judaism_in_Australia, New_Haven_County_Cutters, International_Baseball_League_of_Australia, 2001_Baseball_World_Cup, Northeast_League, Taiwan_Major_League, Masada_College, 2006_Claxton_Shield, 2004_Claxton_Shield, 2007_Claxton_Shield, Catskill_Cougars]]","[[2004 Summer Olympics, Baseball at the Summer Olympics, Australians, Run batted in, Second baseman, Maccabi World Union, Jewish Virtual Library, New South Wales Patriots, Southeastern Louisiana University, Sri Lanka national cricket team, Baseball at the 2004 Summer Olympics, World Baseball Classic, Rodney van Buizen, Brett Roneberg, Glenn Williams, Brendan Kingman, 2007 Baseball World Cup, Greg Jelks, 2005 Baseball World Cup, Judaism in Australia, New Haven County Cutters, International Baseball League of Australia, 2001 Baseball World Cup, Northeast League, Taiwan Major League, Masada College, 2006 Claxton Shield, 2004 Claxton Shield, 2007 Claxton Shield, Catskill Cougars]]","[[[101, 2432, 26

## Collator

In [ ]:
#| export
class XCCollator:

    def __init__(self, tfms):
        self.tfms = tfms

    def __call__(self, x):
        return self.tfms(x)
        

### Example

In [ ]:
tokz = AutoTokenizer.from_pretrained('bert-base-uncased')
tfm_cfg = {
    'sep_tok': tokz.sep_token_id,
    'pad_tok': tokz.pad_token_id,
    'prefix': 'lbl',
    'pad_side': 'right',
    'inp': 'data',
    'targ': 'lbl2data',
    'ptr': 'lbl2data_data2ptr',
    'drop': False,
    'ret_t': True,
    'in_place': False,
    'lev': 0,
    'device': 'cpu',
}

In [ ]:
tfms = TfmPipeline([XCPadFeatTfm(**tfm_cfg), AlignInputIdsTfm(**tfm_cfg)])
collator = XCCollator(tfms)

In [ ]:
b = train_dset.one_batch(10)
o = collator(b)

In [ ]:
o

{'lbl2data_idx': tensor([152270, 152271,  21303,  19438, 259968, 134235, 181495,  22805, 266055,
        138124,  63265, 130051, 195205,  57429,   2567,   2568]), 'lbl2data_identifier': ['Communes_of_the_Alpes-de-Haute-Provence_department', '3=Liste_des_anciennes_communes_des_Alpes-de-Haute-Provence', 'List_of_mountain_peaks_of_the_United_States', 'List_of_mountain_peaks_of_North_America', '2002-03_Heineken_Cup', 'Football_in_Poland', 'PogoÃ\x85Â\x84_Szczecin', 'List_of_national_anthems', 'National_Anthem_of_South_Ossetia', 'VicuÃ\x83Â±a_family', 'Louisiana_African_American_Heritage_Trail', 'Creoles_of_color', 'UK_insolvency_law', 'Communes_of_the_Seine-et-Marne_department', 'List_of_foreign_ministers_in_2017', 'List_of_current_foreign_ministers'], 'lbl2data_input_text': ['Communes of the Alpes-de-Haute-Provence department', '3=Liste des anciennes communes des Alpes-de-Haute-Provence', 'List of mountain peaks of the United States', 'List of mountain peaks of North America', '2002-03 He

In [ ]:
dl = DataLoader(train_dset, batch_size=10, collate_fn=collator)

In [ ]:
b = next(iter(dl))

In [ ]:
b

{'lbl2data_idx': tensor([    0,     2,     3,     9, 26766,    14,    13,    16,    18,    23,
           29,    45,    60,    51,    67,    65,   101,   102]), 'lbl2data_identifier': ['Antinomianism', 'Anarchism', 'Autism', 'Conimbricenses', 'Aristotle', 'List_of_actors_with_Academy_Award_nominations', 'List_of_Academy_Award_records', 'Clock_synchronization', 'Precision_Time_Protocol', 'Egotism', 'Inclusive_fitness', 'Canadian_pioneers_in_early_Hollywood', 'Qualitative_research', 'Ethnobiology', 'History_of_agricultural_science', 'American_Society_of_Agronomy', '12_basic_principles_of_animation', 'Anime'], 'lbl2data_input_text': ['Antinomianism', 'Anarchism', 'Autism', 'Conimbricenses', 'Aristotle', 'List of actors with Academy Award nominations', 'List of Academy Award records', 'Clock synchronization', 'Precision Time Protocol', 'Egotism', 'Inclusive fitness', 'Canadian pioneers in early Hollywood', 'Qualitative research', 'Ethnobiology', 'History of agricultural science', 'American

In [ ]:
b.keys()

dict_keys(['lbl2data_idx', 'lbl2data_identifier', 'lbl2data_input_text', 'lbl2data_input_ids', 'lbl2data_token_type_ids', 'lbl2data_attention_mask', 'lbl2data_data2ptr', 'data_identifier', 'data_input_text', 'data_input_ids', 'data_token_type_ids', 'data_attention_mask', 'hlk2lbl2data_idx', 'hlk2lbl2data_identifier', 'hlk2lbl2data_input_text', 'hlk2lbl2data_input_ids', 'hlk2lbl2data_token_type_ids', 'hlk2lbl2data_attention_mask', 'hlk2lbl2data_data2ptr', 'hlk2lbl2data_lbl2ptr', 'hlk2data_idx', 'hlk2data_identifier', 'hlk2data_input_text', 'hlk2data_input_ids', 'hlk2data_token_type_ids', 'hlk2data_attention_mask', 'hlk2data_data2ptr'])

## Configuration

In [ ]:
import json

In [ ]:
cfg = {
    'path':{
        'train': {
            'data_lbl': '/home/scai/phd/aiz218323/Projects/XC_NLG/data/(mapped)LF-WikiSeeAlsoTitles-320K/trn_X_Y.txt',
            'data_info': '/home/scai/phd/aiz218323/Projects/XC_NLG/data/(mapped)LF-WikiSeeAlsoTitles-320K/raw_data/train.raw.txt',
            'lbl_info': '/home/scai/phd/aiz218323/Projects/XC_NLG/data/(mapped)LF-WikiSeeAlsoTitles-320K/raw_data/label.raw.txt',
            'data_lbl_filterer': '/home/scai/phd/aiz218323/Projects/XC_NLG/data/(mapped)LF-WikiSeeAlsoTitles-320K/filter_labels_train.txt',
            'hlk_meta': {
                'prefix': 'hlk',
                'data_meta': '/home/scai/phd/aiz218323/Projects/XC_NLG/data/(mapped)LF-WikiSeeAlsoTitles-320K/hyper_link_trn_X_Y.txt',
                'lbl_meta': '/home/scai/phd/aiz218323/Projects/XC_NLG/data/(mapped)LF-WikiSeeAlsoTitles-320K/hyper_link_lbl_X_Y.txt',
                'meta_info': '/home/scai/phd/aiz218323/Projects/XC_NLG/data/(mapped)LF-WikiSeeAlsoTitles-320K/raw_data/hyper_link.raw.txt',
            },
        },
        'test': {
            'data_lbl': '/home/scai/phd/aiz218323/Projects/XC_NLG/data/(mapped)LF-WikiSeeAlsoTitles-320K/tst_X_Y.txt',
            'data_info': '/home/scai/phd/aiz218323/Projects/XC_NLG/data/(mapped)LF-WikiSeeAlsoTitles-320K/raw_data/test.raw.txt',
            'lbl_info': '/home/scai/phd/aiz218323/Projects/XC_NLG/data/(mapped)LF-WikiSeeAlsoTitles-320K/raw_data/label.raw.txt',
            'data_lbl_filterer': '/home/scai/phd/aiz218323/Projects/XC_NLG/data/(mapped)LF-WikiSeeAlsoTitles-320K/filter_labels_test.txt',
            'hlk_meta': {
                'prefix': 'hlk',
                'data_meta': '/home/scai/phd/aiz218323/Projects/XC_NLG/data/(mapped)LF-WikiSeeAlsoTitles-320K/hyper_link_tst_X_Y.txt',
                'lbl_meta': '/home/scai/phd/aiz218323/Projects/XC_NLG/data/(mapped)LF-WikiSeeAlsoTitles-320K/hyper_link_lbl_X_Y.txt',
                'meta_info': '/home/scai/phd/aiz218323/Projects/XC_NLG/data/(mapped)LF-WikiSeeAlsoTitles-320K/raw_data/hyper_link.raw.txt',
            },
        },
    },
    'parameters': {
        'cols': ['identifier', 'input_text'],
        'use_tokz': True,
        'tokz': 'bert-base-uncased',
        'fld': 'input_text',
        'max_len': 32,
        'pad_side': 'right',
        'inp': 'data',
        'targ': 'lbl2data',
        'ptr': 'lbl2data_data-ptr',
        'drop': False,
        'ret_t': True,
        'in_place': False,
        'lev': 0,
        'device': 'cpu',
    },
}

In [ ]:
cfg_dir = '/home/scai/phd/aiz218323/scratch/Projects/xcai/cfg/'

In [ ]:
with open(f'{cfg_dir}/data.json', 'w') as f:
    json.dump(cfg, f, indent=2)

## Data block

### BaseXCDataBlock

In [ ]:
#| export
class BaseXCDataBlock:

    @delegates(DataLoader.__init__)
    def __init__(self, 
                 dset:XCDataset, 
                 collate_fn:Callable=None,
                 **kwargs):
        self.dset, self.dl_kwargs, self.collate_fn = dset, self._get_dl_kwargs(**kwargs), collate_fn
        self.dl = DataLoader(dset, collate_fn=collate_fn, **self.dl_kwargs) if collate_fn is not None else None

    @classmethod
    @delegates(XCDataset.from_file)
    def from_file(cls, collate_fn:Callable=None, **kwargs):
        return BaseXCDataBlock(XCDataset.from_file(**kwargs), collate_fn, **kwargs)

    def __len__(self):
        return len(self.dset)

    def _get_dl_kwargs(self, **kwargs):
        dl_params = inspect.signature(DataLoader.__init__).parameters
        return {k:v for k,v in kwargs.items() if k in dl_params}

    
    def _getitems(self, idxs:List):
        return BaseXCDataBlock(self.dset._getitems(idxs), collate_fn=self.collate_fn, **self.dl_kwargs)

    @property
    def bsz(self): return self.dl.batch_size

    @bsz.setter
    def bsz(self, v):
        self.dl_kwargs['batch_size'] = v
        self.dl = DataLoader(self.dset, collate_fn=self.collate_fn, **self.dl_kwargs) if self.collate_fn is not None else None

    @property
    def data_lbl_filterer(self): return self.dset.data.data_lbl_filterer

    @data_lbl_filterer.setter
    def data_lbl_filterer(self, val): self.dset.data.data_lbl_filterer = val

    @typedispatch
    def one_batch(self):
        return next(iter(self.dl))

    @typedispatch
    def one_batch(self, bsz:int):
        self.dl_kwargs['batch_size'] = bsz
        self.dl = DataLoader(self.dset, collate_fn=self.collate_fn, **self.dl_kwargs) if self.collate_fn is not None else None
        return next(iter(self.dl))
        
        

In [ ]:
#| export
@patch
def filterer(cls:BaseXCDataBlock, train:'BaseXCDataBlock', valid:'BaseXCDataBlock', fld:Optional[str]='identifier'):
    train_info, valid_info, lbl_info = train.dset.data.data_info, valid.dset.data.data_info, train.dset.data.lbl_info
    if fld not in train_info: raise ValueError(f'`{fld}` not in `data_info`')
        
    train.data_lbl_filterer, valid_filterer = Filterer.generate(train_info[fld], valid_info[fld], lbl_info[fld], 
                                                                train.dset.data.data_lbl, valid.dset.data.data_lbl)
    _, valid_filterer, idx = Filterer.prune(valid.dset.data.data_lbl, valid_filterer)
    
    valid = valid._getitems(idx)
    valid.data_lbl_filterer = valid_filterer
    
    return train, valid

@patch
def splitter(cls:BaseXCDataBlock, valid_pct:Optional[float]=0.2, seed=None):
    if seed is not None: torch.manual_seed(seed)
    rnd_idx = list(torch.randperm(len(cls)).numpy())
    cut = int(valid_pct * len(cls))
    train, valid = cls._getitems(rnd_idx[cut:]), cls._getitems(rnd_idx[:cut])
    if cls.data_lbl_filterer is None: return train, valid
    else: return cls.filterer(train, valid)

@patch
def sample(cls:BaseXCDataBlock, pct:Optional[float]=0.2, n:Optional[int]=None, seed=None):
    if seed is not None: torch.manual_seed(seed)
    rnd_idx = list(torch.randperm(len(cls)).numpy())
    cut = int(pct * len(cls)) if n is None else max(1, n)
    return cls._getitems(rnd_idx[:cut])
    

#### Example

In [ ]:
train_block = BaseXCDataBlock(train_dset, batch_size=2, collate_fn=collator)

In [ ]:
split1, split2 = train_block.splitter()
print('split 1:',split1.dset.n_data,split1.dset.n_lbl,split1.dset.n_meta)
print('split 2:',split2.dset.n_data,split2.dset.n_lbl,split2.dset.n_meta) 

/scratch/scai/phd/aiz218323/anaconda3/envs/xc_nlg/lib/python3.9/site-packages/scipy/sparse/_index.py:145: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


split 1: 554466 312330 2458399
split 2: 137578 312330 2458399


In [ ]:
sample = train_block.sample(n=40)
print(sample.dset.n_data,sample.dset.n_lbl,sample.dset.n_meta)

40 312330 2458399


In [ ]:
b = train_block.one_batch()

In [ ]:
b = train_block.one_batch(20)

In [ ]:
train_block = BaseXCDataBlock.from_file(**cfg['path']['train'], **cfg['parameters'], collate_fn=collator)

/home/scai/phd/aiz218323/.local/lib/python3.9/site-packages/xclib-0.97-py3.9-linux-x86_64.egg/xclib/data/data_utils.py:263: UserWarning: Header mis-match from inferred shape!
  warnings.warn("Header mis-match from inferred shape!")


In [ ]:
next(iter(train_block.dl))

{'lbl2data_idx': tensor([0, 1, 2]), 'lbl2data_identifier': ['Antinomianism', 'Libertarian_socialism', 'Anarchism'], 'lbl2data_input_text': ['Antinomianism', 'Libertarian socialism', 'Anarchism'], 'lbl2data_input_ids': tensor([[  101,  3424,  3630, 20924,   102,     0],
        [  101, 19297, 14649,   102,     0,     0],
        [  101,  9617, 11140,  2964,   102,     0]]), 'lbl2data_token_type_ids': tensor([[0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0]]), 'lbl2data_attention_mask': tensor([[1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 0]]), 'lbl2data_data2ptr': tensor([3]), 'data_identifier': ['Anarchism'], 'data_input_text': ['Anarchism'], 'data_input_ids': tensor([[  101,  9617, 11140,  2964,   102]]), 'data_token_type_ids': tensor([[0, 0, 0, 0, 0]]), 'data_attention_mask': tensor([[1, 1, 1, 1, 1]]), 'hlk2lbl2data_idx': tensor([   1992,    4612,    4697,    4706,    4709,    4892,    5256,    7587,
           9813,   10948,   110

### XCDataBlock

In [ ]:
#| export
class XCDataBlock:

    def __init__(self, train:BaseXCDataBlock=None, valid:BaseXCDataBlock=None, test:BaseXCDataBlock=None):
        self.train, self.valid, self.test = train, valid, test

    @staticmethod
    def load_cfg(fname):
        with open(fname, 'r') as f: return json.load(f)

    @property
    def lbl_info(self): return self.train.dset.data.lbl_info

    @property
    def lbl_dset(self): return MainXCDataset(self.train.dset.data.lbl_info)

    @property
    def n_lbl(self): return self.train.dset.n_lbl

    @property
    def collator(self): return self.train.collate_fn

    def sample_info(self, info, idx):
        return {k: v[idx] if isinstance(v, torch.Tensor) or isinstance(v, np.ndarray) else [v[i] for i in idx] for k,v in info.items()}

    def linker_dset(self, meta_name:str, remove_empty:Optional[bool]=True):
        if meta_name not in self.train.dset.meta:
            raise ValueError(f'Invalid metadata: {meta_name}')

        train_meta = self.train.dset.meta[meta_name].data_meta
        if remove_empty:
            train_idx = np.where(train_meta.getnnz(axis=1) > 0)[0]
            meta_idx = np.where(train_meta.getnnz(axis=0) > 0)[0]
            train_meta = train_meta[train_idx][:, meta_idx].tocsr()
        
        train_info = self.train.dset.data.data_info
        meta_info = self.train.dset.meta[meta_name].meta_info

        if remove_empty:
            train_info = self.sample_info(train_info, train_idx)
            meta_info = self.sample_info(meta_info, meta_idx)

        collate_fn = self.train.collate_fn
        train_dset = BaseXCDataBlock(XCDataset(MainXCDataset(data_info=train_info, data_lbl=train_meta, lbl_info=meta_info)), 
                                     collate_fn=collate_fn)
        
        if self.test is not None:
            test_meta = self.test.dset.meta[meta_name].data_meta
            if remove_empty:
                test_meta = test_meta[:, meta_idx].tocsr()
                test_idx = np.where(test_meta.getnnz(axis=1) > 0)[0]
                test_meta = test_meta[test_idx].tocsr()
    
            test_info = self.test.dset.data.data_info
            if remove_empty: test_info = self.sample_info(test_info, test_idx)
    
            test_dset = BaseXCDataBlock(XCDataset(MainXCDataset(data_info=test_info, data_lbl=test_meta, lbl_info=meta_info)), 
                                        collate_fn=collate_fn)
            return XCDataBlock(train=train_dset, test=test_dset)
        
        return XCDataBlock(train=train_dset)

    def inference_dset(self, data_info:Dict, data_lbl:sparse.csr_matrix, lbl_info:Dict, data_lbl_filterer, **kwargs):
        x_idx = np.where(data_lbl.getnnz(axis=1) == 0)[0].reshape(-1,1)
        y_idx = np.zeros((len(x_idx),1), dtype=np.int64)
        data_lbl[x_idx, y_idx] = 1
        data_lbl_filterer = np.vstack([np.hstack([x_idx, y_idx]), data_lbl_filterer])
    
        pred_dset = XCDataset(MainXCDataset(data_info=data_info, data_lbl=data_lbl, lbl_info=lbl_info,
                                            data_lbl_filterer=data_lbl_filterer, **kwargs))
        return pred_dset
        
    @classmethod
    def from_cfg(cls, 
                 cfg:Union[str,Dict],
                 collate_fn:Optional[Callable]=None,
                 valid_pct:Optional[float]=0.2,
                 seed=None):
        if isinstance(cfg, str): cfg = cls.load_cfg(cfg)
        blks = {o:BaseXCDataBlock.from_file(**cfg['path'][o], **cfg['parameters'], collate_fn=collate_fn) for o in ['train', 'valid', 'test'] if o in cfg['path']}
        # if 'valid' not in blks: blks['train'], blks['valid'] = blks['train'].splitter(valid_pct, seed=seed)
        return cls(**blks)
        

#### Example 1

In [ ]:
block = XCDataBlock(train=train_block)

In [ ]:
lbl_info = block.lbl_info

In [ ]:
lbl_info['input_ids'][:10]

[[101, 3424, 3630, 20924, 2964, 102],
 [101, 19297, 14649, 102],
 [101, 9617, 11140, 2964, 102],
 [101, 19465, 102],
 [101, 13798, 1997, 8181, 5367, 102],
 [101, 7734, 2162, 1997, 6889, 102],
 [101, 4519, 2793, 5349, 102],
 [101, 2862, 1997, 7008, 1997, 8181, 5367, 102],
 [101, 2862, 1997, 2942, 2916, 4177, 102],
 [101, 9530, 5714, 23736, 19023, 2229, 102]]

In [ ]:
%time block = XCDataBlock.from_cfg(cfg, collate_fn=collator)

CPU times: user 1min 18s, sys: 930 ms, total: 1min 19s
Wall time: 44.2 s


In [ ]:
print('n_data :', f'{block.train.dset.n_data},{block.valid.dset.n_data},{block.test.dset.n_data}')
print('n_lbl  :', f'{block.train.dset.n_lbl},{block.valid.dset.n_lbl},{block.test.dset.n_lbl}')
print('n_meta :', f'{block.train.dset.n_meta},{block.valid.dset.n_meta},{block.test.dset.n_meta}')

n_data : 554466,138616,177515
n_lbl  : 312330,312330,312330
n_meta : None,None,None


In [ ]:
block.valid.dset.show_data(n=2)

,data_identifier,data_input_text,data_input_ids,data_token_type_ids,data_attention_mask,lbl2data_idx,lbl2data_identifier,lbl2data_input_text,lbl2data_input_ids,lbl2data_token_type_ids,lbl2data_attention_mask,hlk2data_idx,hlk2data_identifier,hlk2data_input_text,hlk2data_input_ids,hlk2data_token_type_ids,hlk2data_attention_mask,hlk2lbl2data_idx,hlk2lbl2data_identifier,hlk2lbl2data_input_text,hlk2lbl2data_input_ids,hlk2lbl2data_token_type_ids,hlk2lbl2data_attention_mask
0,2019_Caribbean_Club_Championship,2019 Caribbean Club Championship,"[101, 10476, 7139, 2252, 2528, 102]","[0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1]","[310617, 310618]","[2019_CONCACAF_League, 2020_CONCACAF_Champions_League]","[2019 CONCACAF League, 2020 CONCACAF Champions League]","[[101, 10476, 22169, 2223, 102], [101, 12609, 22169, 3966, 2223, 102]]","[[0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]]","[[1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]","[2932, 4843, 8132, 115570, 122131, 147896, 158253, 158260, 246100, 286923, 289314, 304800, 350441, 403932, 416728, 416741, 416746, 416750, 416921, 416926, 416929, 416930, 416945, 416949, 416954, 416961, 427446, 427447, 427448, 427449, 427450, 427451, 427452, 427453, 427454, 427455, 427456, 427457, 427458, 427459, 427460, 427461]","[California, San_Francisco, Eastern_Time_Zone, Willemstad, CONCACAF, Jamaica_Football_Federation, Football_Association_of_Cuba, Bermuda_Football_Association, Caribbean_Football_Union, Surinamese_Football_Association, Guyana_Football_Federation, Barbados_Football_Association, Caribbean_Club_Championship, Bahamas_Football_Association, Cibao_FC, Grenada_Football_Association, Club_Franciscain, Ligue_de_football_de_la_Martinique, Weymouth_Wales_FC, Anguilla_Football_Association, Aruba_Football_Federation, SV_Real_Rincon, Bonaire_Football_Federation, Dominica_Football_Association, Guadeloupean_League_of_Football, Montserrat_Football_Association, 2017_TT_Pro_League, AS_Capoise, 2018_Caribbean_Club_Championship, SV_Dakota, Fruta_Conquerors_FC, FC_Santiago_de_Cuba, Platinum_FC, 2018_SLFA_First_Division, Village_Superstars_FC, 2020_Caribbean_Club_Championship, Ergilio_Hato_Stadium, 2018_Campeonato_Nacional_de_FÃºtbol_de_Cuba, 2018_Barbados_Premier_League, Scholars_International_SC, CRKSV_Jong_Holland, CS_Moulien]","[California, San Francisco, Eastern Time Zone, Willemstad, CONCACAF, Jamaica Football Federation, Football Association of Cuba, Bermuda Football Association, Caribbean Football Union, Surinamese Football Association, Guyana Football Federation, Barbados Football Association, Caribbean Club Championship, Bahamas Football Association, Cibao FC, Grenada Football Association, Club Franciscain, Ligue de football de la Martinique, Weymouth Wales FC, Anguilla Football Association, Aruba Football Federation, SV Real Rincon, Bonaire Football Federation, Dominica Football Association, Guadeloupean League of Football, Montserrat Football Association, 2017 TT Pro League, AS Capoise, 2018 Caribbean Club Championship, SV Dakota, Fruta Conquerors FC, FC Santiago de Cuba, Platinum FC, 2018 SLFA First Division, Village Superstars FC, 2020 Caribbean Club Championship, Ergilio Hato Stadium, 2018 Campeonato Nacional de FÃºtbol de Cuba, 2018 Barbados Premier League, Scholars International SC, CRKSV Jong Holland, CS Moulien]","[[101, 2662, 102], [101, 2624, 3799, 102], [101, 2789, 2051, 4224, 102], [101, 18811, 16917, 102], [101, 22169, 102], [101, 9156, 2374, 4657, 102], [101, 2374, 2523, 1997, 7394, 102], [101, 13525, 2374, 2523, 102], [101, 7139, 2374, 2586, 102], [101, 25050, 3366, 2374, 2523, 102], [101, 18786, 2374, 4657, 102], [101, 16893, 2374, 2523, 102], [101, 7139, 2252, 2528, 102], [101, 17094, 2374, 2523, 102], [101, 25022, 3676, 2080, 4429, 102], [101, 29153, 2374, 2523, 102], [101, 2252, 4557, 3540, 2378, 102], [101, 18374, 2139, 2374, 2139, 2474, 29365, 102], [101, 2057, 24335, 17167, 3575, 4429, 102], [101, 17076, 19231, 2721, 2374, 2523, 102], [101, 12098, 19761, 2374, 4657, 102], [101, 17917, 2613, 15544, 15305, 

In [ ]:
train_lbl = block.train.dset.data.data_lbl
test_lbl = block.valid.dset.data.data_lbl

In [ ]:
train_identifier = block.train.dset.data.data_info['identifier']
test_identifier = block.valid.dset.data.data_info['identifier']
lbl_identifier = block.train.dset.data.lbl_info['identifier']

In [ ]:
x, y = gen_filterer(train_identifier, test_identifier, lbl_identifier, train_lbl, test_lbl)

#### Example 2

In [ ]:
lbl_dset = block.lbl_dset

In [ ]:
lbl_dset[100]

{'data_identifier': 'Alt_codes',
 'data_input_text': 'Alt codes',
 'data_input_ids': [101, 12456, 9537, 102],
 'data_token_type_ids': [0, 0, 0, 0],
 'data_attention_mask': [1, 1, 1, 1]}